<a href="https://colab.research.google.com/github/ANISHG-26/ELG-5142-Project-Group-6/blob/main/ELG_5142_Accident_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import seaborn as sns
from yellowbrick.target import FeatureCorrelation
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [2]:
original_data_directory = "ELG-5142-Project-Group-6/"

if not(os.path.exists(original_data_directory.split("/")[0])):
  !git clone https://github.com/ANISHG-26/ELG-5142-Project-Group-6.git
os.chdir(original_data_directory)

Cloning into 'ELG-5142-Project-Group-6'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 3 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), 994.67 KiB | 6.67 MiB/s, done.


In [3]:
data = "2019_Tabular_Transportation_Collision_Data.csv"
df = pd.read_csv(data)
df

,Anom_ID,Accident_Date,Accident_Time,Location,Geo_ID,Accident_Location,Classification_of_Accident,Initial_Impact_Type,Environment_Condition,Light,...,No__of_Injuries,No__of_Minimal,No__of_Minor,No__of_Major,No__of_Fatal,X,Y,Latitude,Longitude,ObjectId
0,19--10709,2019/09/04,7:40 AM,HWY 417 WALKLEY IC110R57 @ WALKLEY EXTENSION R...,4164,03 - At intersection,03 - P.D. only,02 - Angle,02 - Rain,01 - Daylight,...,0,NaN,NaN,NaN,NaN,375656.9048,5028998.541,45.397340,-75.594967,1
1,19--10079,2019/08/19,9:08 PM,GREENBANK RD @ MARKETPLACE AVE (0010406),10406,02 - Intersection related,03 - P.D. only,03 - Rear end,01 - Clear,07 - Dark,...,0,NaN,NaN,NaN,NaN,364008.2994,5014602.241,45.268885,-75.745459,2
2,19--1071,2019/01/20,5:16 PM,HUNT CLUB RD @ UPLANDS DR (0010844),10844,03 - At intersection,02 - Non-fatal injury,07 - SMV other,01 - Clear,05 - Dusk,...,1,1.0,NaN,NaN,NaN,370156.8471,5023121.493,45.344995,-75.665988,3
3,19--1008,2019/01/20,10:00 PM,ST. JOSEPH BLVD @ TENTH LINE RD (0009163),9163,02 - Intersection related,03 - P.D. only,07 - SMV other,03 - Snow,07 - Dark,...,0,NaN,NaN,NaN,NaN,382890.9175,5038666.445,45.483554,-75.501049,4
4,19--10710,2019/09/04,8:00 AM,BAYVIEW RD btwn BURNSIDE AVE & SCOTT ST (__3ZA...,__3ZA0V3,04 - At/near private drive,02 - Non-fatal injury,05 - Turning movement,01 - Clear,01 - Daylight,...,1,1.0,NaN,NaN,NaN,365212.0509,5030227.695,45.409378,-75.728212,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16394,19--9995,2019/08/17,6:48 PM,HAMILTON AVE N btwn WELLINGTON ST W & TYNDAL...,__3ZA03I,04 - At/near private drive,03 - P.D. only,06 - SMV unattended vehicle,02 - Rain,01 - Daylight,...,0,NaN,NaN,NaN,NaN,365175.9574,5029006.170,45.398391,-75.728823,16395
16395,19--9996,2019/08/17,7:00 PM,TRIM RD @ DAIRY DR/TAYLOR CREEK DR (0012921),12921,03 - At intersection,03 - P.D. only,99 - Other,01 - Clear,01 - Daylight,...,0,NaN,NaN,NaN,NaN,384534.9920,5039595.393,45.491726,-75.479870,16396
16396,19--9997,2019/08/17,7:33 PM,NORTH RIVER RD @ SELKIRK ST (0009804),9804,02 - Intersection related,03 - P.D. only,07 - SMV other,01 - Clear,01 - Daylight,...,0,NaN,NaN,NaN,NaN,369780.6920,5032758.790,45.431743,-75.669518,16397
16397,19--9998,2019/08/17,7:49 AM,GRENADIER WAY btwn QUEENSBURY DR & FIELDCREST ...,__3ZA599,01 - Non intersection,03 - P.D. only,06 - SMV unattended vehicle,01 - Clear,00 - Unknown,...,0,NaN,NaN,NaN,NaN,365654.0573,5016221.621,45.283316,-75.724289,16398


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16399 entries, 0 to 16398
Data columns (total 28 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Anom_ID                     16399 non-null  object 
 1   Accident_Date               16399 non-null  object 
 2   Accident_Time               16399 non-null  object 
 3   Location                    16399 non-null  object 
 4   Geo_ID                      16399 non-null  object 
 5   Accident_Location           16399 non-null  object 
 6   Classification_of_Accident  16399 non-null  object 
 7   Initial_Impact_Type         16399 non-null  object 
 8   Environment_Condition       16399 non-null  object 
 9   Light                       16399 non-null  object 
 10  Road_Surface_Condition      16399 non-null  object 
 11  Traffic_Control             16399 non-null  object 
 12  Traffic_Control_Condition   8735 non-null   object 
 13  No__of_Vehicles             163

In [5]:
df.head().T

,0,1,2,3,4
Anom_ID,19--10709,19--10079,19--1071,19--1008,19--10710
Accident_Date,2019/09/04,2019/08/19,2019/01/20,2019/01/20,2019/09/04
Accident_Time,7:40 AM,9:08 PM,5:16 PM,10:00 PM,8:00 AM
Location,HWY 417 WALKLEY IC110R57 @ WALKLEY EXTENSION R...,GREENBANK RD @ MARKETPLACE AVE (0010406),HUNT CLUB RD @ UPLANDS DR (0010844),ST. JOSEPH BLVD @ TENTH LINE RD (0009163),BAYVIEW RD btwn BURNSIDE AVE & SCOTT ST (__3ZA...
Geo_ID,4164,10406,10844,9163,__3ZA0V3
Accident_Location,03 - At intersection,02 - Intersection related,03 - At intersection,02 - Intersection related,04 - At/near private drive
Classification_of_Accident,03 - P.D. only,03 - P.D. only,02 - Non-fatal injury,03 - P.D. only,02 - Non-fatal injury
Initial_Impact_Type,02 - Angle,03 - Rear end,07 - SMV other,07 - SMV other,05 - Turning movement
Environment_Condition,02 - Rain,01 - Clear,01 - Clear,03 - Snow,01 - Clear
Light,01 - Daylight,07 - Dark,05 - Dusk,07 - Dark,01 - Daylight


In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
No__of_Vehicles,16399.0,1.862736e+00,0.564858,1.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,8.000000e+00
No__of_Bicycles,16399.0,1.811086e-02,0.134268,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00
No__of_Motorcycles,16399.0,8.110251e-03,0.090371,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00
No__of_Pedestrians,16399.0,1.920849e-02,0.142064,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00
No__of_Injuries,16399.0,2.197695e-01,0.634609,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.800000e+01
No__of_Minimal,1198.0,1.195326e+00,0.579728,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.100000e+01
No__of_Minor,1575.0,1.276825e+00,0.672241,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+01
No__of_Major,108.0,1.240741e+00,1.303270,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.400000e+01
No__of_Fatal,25.0,1.080000e+00,0.400000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.000000e+00
X,16399.0,3.670292e+05,9339.765281,3.174750e+05,3.633117e+05,3.677075e+05,3.716748e+05,4.012228e+05
